<a href="https://colab.research.google.com/github/BeatrixBlaine/DS-C1/blob/main/3_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def init_cognitive_state():
    return {
        "+": 0.0,
        "-": 0.0,
        "×": 0.0
    }


In [2]:
# generator

import random

def generate_demo():
    ops = ["+", "-", "×"]
    a, b = random.randint(1, 20), random.randint(1, 20)
    op = random.choice(ops)

    if op == "+":
        ans = a + b
    elif op == "-":
        ans = a - b
    else:
        ans = a * b

    return {
        "question": f"What is {a} {op} {b}?",
        "answer": str(ans),
        "op": op
    }


In [5]:
# increase confindence in operation

def update_cognitive_state(state, demo, alpha=0.3):
    op = demo["op"]
    state[op] += alpha
    return state


In [6]:
# score function

def csass_score(demo, query_op, state):
    relevance = 1.0 if demo["op"] == query_op else 0.3
    need = 1.0 - state[demo["op"]]   # select what the model "needs"
    return relevance * need


In [4]:
# selection algorithm

def csass_select(pool, query, k=4):
    state = init_cognitive_state()
    selected = []

    query_op = next(op for op in ["+", "-", "×"] if op in query)

    for _ in range(k):
        scored = []
        for demo in pool:
            if demo not in selected:
                score = csass_score(demo, query_op, state)
                scored.append((score, demo))

        scored.sort(reverse=True, key=lambda x: x[0])
        best_demo = scored[0][1]

        selected.append(best_demo)
        state = update_cognitive_state(state, best_demo)

    return selected


In [7]:
# prompt builder

def build_prompt(demos, query):
    prompt = "You are a grade school math solver.\n\n"
    for d in demos:
        prompt += f"Q: {d['question']}\nA: {d['answer']}\n\n"
    prompt += f"Q: {query}\nA:"
    return prompt


In [8]:
# Build synthetic candidate pool
pool = [generate_demo() for _ in range(40)]

query = "What is 7 × 8?"

# CSASS selection
demos = csass_select(pool, query, k=4)

prompt = build_prompt(demos, query)
print(prompt)


You are a grade school math solver.

Q: What is 10 × 4?
A: 40

Q: What is 12 × 15?
A: 180

Q: What is 4 × 5?
A: 20

Q: What is 13 - 14?
A: -1

Q: What is 7 × 8?
A:
